In [12]:
import psycopg2
from psycopg2 import sql

# Database connection settings
db_settings = {
    'dbname': 'wikiknowledge',
    'user': 'ocelot',
    'password': 'odinson',
    'host': 'localhost'
}

def get_vector_by_document(conn, document_name: str):
    """Retrieve the vector from the PostgreSQL database using the document name."""
    with conn.cursor() as cur:
        try:
            cur.execute(sql.SQL("SELECT reduced_embedding FROM my_vectors_reduced WHERE document = %s"), [document_name])
            result = cur.fetchone()
            if result:
                return result[0]
            else:
                print(f"No vector found for document: {document_name}")
                conn.rollback()  # Rollback if no document is found
                return None
        except Exception as e:
            print(f"Error querying for document '{document_name}': {e}")
            conn.rollback()
            return None

def query_db(conn, query_vector):
    """Query the PostgreSQL database for the nearest neighbors using the given vector."""
    with conn.cursor() as cur:
        try:
            cur.execute(sql.SQL("""
                SELECT document, (reduced_embedding <-> %s) AS distance
                FROM my_vectors_reduced
                ORDER BY distance ASC
                LIMIT 20
            """), [query_vector])
            results = cur.fetchall()
            return results
        except Exception as e:
            print(f"Error querying database: {e}")
            conn.rollback()
            return []

# Connect to PostgreSQL
conn = psycopg2.connect(**db_settings)


In [13]:
# Document name to search
document_name = "Anthony Wayne"

# Get the vector from the database
query_vector = get_vector_by_document(conn, document_name)
print("We have the query vector")

if query_vector:
    # Query the database using the vector
    results = query_db(conn, query_vector)
    
    print(f"Query results for document '{document_name}':")
    for document, distance in results:
        print(f"Document: {document}, Distance: {distance}")


We have the query vector
Query results for document 'Anthony Wayne':
Document: Anthony Wayne, Distance: 0.0
Document: Francis Marion, Distance: 0.6210231860880239
Document: Jeffery Amherst, 1st Baron Amherst, Distance: 0.6356060860969648
Document: Casimir Pulaski, Distance: 0.6564066563681961
Document: Winfield Scott, Distance: 0.6852709440407648
Document: Redlining, Distance: 0.7123884967088341
Document: Turion X2 Ultra, Distance: 0.7150777631767272
Document: IBM PS/2 Note, Distance: 0.7156248567406124
Document: Generali Arena (Vienna), Distance: 0.7175636867343594
Document: Cati, Distance: 0.7216879053125443
Document: Martin Hristov, Distance: 0.7234277679250143
Document: Lanzhou class destroyer, Distance: 0.7263470402031591
Document: FCW, Distance: 0.7265556089012923
Document: Buick, Distance: 0.7274725352343301
Document: Gujarati-language, Distance: 0.7279091117945765
Document: 6062 Vespa, Distance: 0.7280715939168915
Document: Ford Ecoboost engine, Distance: 0.7282885486471946
Doc

In [4]:
conn.close()